In [ ]:
import torch

act_rs = torch.load("final_result/nvfp4_rs/extractions/activations_during_generation.pt", weights_only=False)
w_rs_af = torch.load("final_result/nvfp4_rs/extractions/weights_after_smooth.pt", weights_only=False)

act_org = torch.load("final_result/nvfp4/extractions/activations_during_generation.pt", weights_only=False)
w_org = torch.load("final_result/nvfp4/extractions/weights_before_smooth.pt", weights_only=False)

In [ ]:
w_rs_af.keys()

In [ ]:
smooth_scale = torch.load("runs/diffusion/cache/quant/qdiff.128/smooth/w.4-x.4-y.16-w.4/w.sfp4_e2m1_all-x.sfp4_e2m1_all-y.bf16-w.sint4/w.v16.sfp8_e4m3_nan.tsnr.bf16-x.v16.sfp8_e4m3_nan-y.tnsr.bf16-w.v64.bf16/w.skip.[e+rs+rtp+s+tpi+tpo]-x.skip.[e+rs+rtp+s+tan+tn+tpi+tpo]-y.skip.[]-w.include.[tan+tn]/smooth.proj.OutputsError.GridSearch.Layer.d2.en1.sn1/smooth.proj.[a.AbsMax.b.AbsMax]/smooth.proj.g20.a0.bn2/smooth.proj..rev.nf/smooth.proj.skip.[rc+tan+tn]/flux.1-schnell.pt", weights_only=False)

In [ ]:
target_layer = 'transformer_blocks.11.attn.to_q'
w = w_org[target_layer]['weight']  # [num_samples, num_timesteps, batch_size, hidden_dim, hidden_dim]
rs_w = w_rs_af[target_layer]['weight']  # [num_samples, num_timesteps, batch_size, hidden_dim, hidden_dim]
act = {
    "MJHQ" : {
        "org" : {
            t : {"input_acts" : [], "output_acts" : []} for t in range(4)
        },
        "reverse" : {
            t : {"input_acts" : [], "output_acts" : []} for t in range(4)
        },
    },
    "sDCI" : {
        "org" : {
            t : {"input_acts" : [], "output_acts" : []} for t in range(4)
        },
        "reverse" : {
            t : {"input_acts" : [], "output_acts" : []} for t in range(4)
        },
    }
}
for i, (ipt, opt) in enumerate(zip(act_org[target_layer+".input"][-1], act_org[target_layer+".output"][-1])):
    if i < 64:
        dataset = "MJHQ"
    else:
        dataset = "sDCI"
    timestep = i % 4
    act[dataset]["org"][timestep]["input_acts"].append(ipt)
    act[dataset]["org"][timestep]["output_acts"].append(opt)
for i, (ipt, opt) in enumerate(zip(act_rs[target_layer+".input"][-1], act_rs[target_layer+".output"][-1])):
    if i < 64:
        dataset = "MJHQ"
    else:
        dataset = "sDCI"
    timestep = i % 4
    act[dataset]["reverse"][timestep]["input_acts"].append(ipt)
    act[dataset]["reverse"][timestep]["output_acts"].append(opt)
for dataset in act.keys():
    for mode in act[dataset].keys():
        for t in act[dataset][mode].keys():
            act[dataset][mode][t]["input_acts"] = torch.stack(act[dataset][mode][t]["input_acts"], dim=0)
            act[dataset][mode][t]["output_acts"] = torch.stack(act[dataset][mode][t]["output_acts"], dim=0)

In [ ]:
torch.save({
    "weights_before_smooth" : w,
    "weights_after_smooth" : rs_w,
    "activations" : act,
}, "investigate_activations_weights.pt")

In [ ]:
import torch

In [ ]:
cache = torch.load("investigate_activations_weights.pt", weights_only=False)
w = cache["weights_before_smooth"]
rs_w = cache["weights_after_smooth"]
act = cache["activations"]
smooth_scale = torch.load("runs/diffusion/cache/quant/qdiff.128/smooth/w.4-x.4-y.16-w.4/w.sfp4_e2m1_all-x.sfp4_e2m1_all-y.bf16-w.sint4/w.v16.sfp8_e4m3_nan.tsnr.bf16-x.v16.sfp8_e4m3_nan-y.tnsr.bf16-w.v64.bf16/w.skip.[e+rs+rtp+s+tpi+tpo]-x.skip.[e+rs+rtp+s+tan+tn+tpi+tpo]-y.skip.[]-w.include.[tan+tn]/smooth.proj.OutputsError.GridSearch.Layer.d2.en1.sn1/smooth.proj.[a.AbsMax.b.AbsMax]/smooth.proj.g20.a0.bn2/smooth.proj..rev.nf/smooth.proj.skip.[rc+tan+tn]/flux.1-schnell.pt", weights_only=False)
smooth_scale = smooth_scale['transformer_blocks.11.attn.to_q']

In [ ]:
@torch.no_grad()
def lzs(x: torch.Tensor, bits: int = 4) -> torch.Tensor:
    """
    Real per-block LZS (shared flag per block).
    Expects x shaped [..., block_size]. Computes a single flag per block from max MSB in the block.

    For bits=4, we keep a 3-bit mantissa [0..7] and a shared power-of-two shift per block.
    """
    # sign + magnitude
    sign = torch.sign(x)
    a = torch.abs(x).clamp_max_(127)  # keep in INT8-like magnitude range

    # MSB per element using frexp: MSB = exp for a>0 else 0
    _, exp = torch.frexp(a)
    msb = torch.where(a > 0, exp, torch.zeros_like(exp))  # same shape as a

    # Block-wise max MSB (shared across last dim = block_size)
    msb_blk = msb.amax(dim=-1, keepdim=True)

    # Shared FLAG per block: max(MSB) - 3, clamped at 0
    flag = (msb_blk - 3).clamp_min_(0)

    # scale = 2^flag (shared per block)
    scale = torch.ldexp(torch.ones_like(a), flag)

    # quantize each element using shared scale
    q = torch.round(a / scale).clamp_(0, 7) * scale
    return sign * q

@torch.no_grad()
def quartz(
    tensor: torch.Tensor,
    quant_block_size: int = 32,
    lzs_block_size: int = 16,
    mode: str = "mx",
) -> torch.Tensor:
    """
    GPU-efficient QUARTZ-like quantization + LZS + dequant (verification path).
    """
    if tensor.numel() == 0:
        return tensor

    orig_shape = tensor.shape
    device = tensor.device
    dtype = tensor.dtype

    # Flatten
    x = tensor.reshape(-1)

    # Pad to multiple of quant_block_size (avoid torch.cat when possible)
    n = x.numel()
    r = n % quant_block_size
    if r != 0:
        pad = quant_block_size - r
        x_pad = torch.empty(n + pad, device=device, dtype=dtype)
        x_pad[:n].copy_(x)
        x_pad[n:].zero_()
        x = x_pad

    # [num_qblocks, quant_block_size]
    x = x.view(-1, quant_block_size)

    # Per-block scale (keep in FP32 for stability; cast back later)
    x_fp = x.float()

    if mode == "mx":
        scales = torch.amax(x_fp.abs(), dim=1, keepdim=True) / 127.0
    elif mode == "lzs":
        # Your original denominator was (2^(8-2)-1)=63 (effectively 7-bit signed w/ extra headroom)
        scales = (torch.amax(x_fp, dim=1, keepdim=True) - torch.amin(x_fp, dim=1, keepdim=True)) / 64.0
    else:
        raise ValueError(f"Unknown mode {mode} for QUARTZ quantization")

    # Guard against zeros (important for all-zero blocks)
    scales = scales.clamp_min_(1e-12)

    # INT8 quant (kept as float here to match your code path)
    q = torch.round(x_fp / scales).clamp_(-128, 127)

    # LZS over all lzs blocks in one shot
    q = q.view(-1, lzs_block_size)
    q = lzs(q, bits=4)
    q = q.view(-1, quant_block_size)

    # Dequant (verification)
    deq = (q * scales).to(dtype)

    # Unpad back to original length, reshape
    if r != 0:
        deq = deq.view(-1)[:n]
    return deq.view(orig_shape)

In [ ]:
# -----------------------------
# FP4 E2M1 quant/dequant (NVFP4 uses E2M1)   [oai_citation:1‡NVIDIA Developer](https://developer.nvidia.com/blog/introducing-nvfp4-for-efficient-and-accurate-low-precision-inference/)
# -----------------------------
@torch.no_grad()
def fp4_e2m1_quant_dequant(x: torch.Tensor) -> torch.Tensor:
    """
    Quantize+dequantize to FP4 E2M1 (bias=1) in float, vectorized.
    Representable magnitudes: {0, 0.5, 1, 1.5, 2, 3, 4, 6} (and negatives).
    Saturates to +/-6.
    """
    sign = torch.sign(x)
    a = torch.abs(x)

    # Saturate to max representable magnitude (~6)
    a = a.clamp_max(6.0)

    out = torch.zeros_like(a)

    # Subnormal/zero region: representable are 0 and 0.5
    # Midpoint thresholds: 0.25 between 0 and 0.5, 0.75 between 0.5 and 1.0
    sub = a < 1.0
    out = torch.where(sub & (a >= 0.25) & (a < 0.75), torch.full_like(a, 0.5), out)
    out = torch.where(sub & (a >= 0.75), torch.ones_like(a), out)  # near 1.0 maps to 1.0

    # Normal region (>=1): values are base * {1.0, 1.5}, base in {1,2,4}
    normal = a >= 1.0
    if normal.any():
        # exponent = floor(log2(a)), clamped to [0,2]
        exp = torch.floor(torch.log2(a.clamp_min(1e-30))).to(torch.int32)
        exp = exp.clamp(0, 2)

        base = torch.pow(torch.tensor(2.0, device=a.device, dtype=a.dtype), exp.to(a.dtype))
        mant = a / base  # in [1,2)

        # 1 mantissa bit => choose 1.0 vs 1.5; threshold at midpoint 1.25
        mant_bit = (mant >= 1.25).to(a.dtype)
        val = base * (1.0 + 0.5 * mant_bit)

        out = torch.where(normal, val, out)

    return sign * out


# -----------------------------
# FP8 E4M3 quant/dequant for block scales   [oai_citation:2‡NVIDIA Developer](https://developer.nvidia.com/blog/introducing-nvfp4-for-efficient-and-accurate-low-precision-inference/)
# -----------------------------
@torch.no_grad()
def fp8_e4m3_quant_dequant(x: torch.Tensor) -> torch.Tensor:
    """
    Approximate quant+dequant to FP8 E4M3 in float, vectorized.

    This is an emulation intended for NVFP4 block scales (E4M3). NVFP4 uses
    an FP8 (E4M3) scale per 16-value block and an FP32 per-tensor scale.  [oai_citation:3‡NVIDIA Developer](https://developer.nvidia.com/blog/introducing-nvfp4-for-efficient-and-accurate-low-precision-inference/)

    We clamp to ~448 max magnitude (common E4M3FN max cited in NVFP4 contexts).  [oai_citation:4‡NVIDIA Docs](https://docs.nvidia.com/deeplearning/tensorrt/latest/inference-library/work-quantized-types.html?utm_source=chatgpt.com)
    """
    sign = torch.sign(x)
    a = torch.abs(x)

    # clamp to finite max (commonly cited as 448 for E4M3 scales in NVFP4 tooling)
    a = a.clamp_max(448.0)

    # Handle zeros
    out = torch.zeros_like(a)
    nz = a > 0
    if not nz.any():
        return out

    # E4M3: bias=7, mantissa bits=3
    bias = 7
    m_bits = 3

    # Use frexp for stable exponent/mantissa extraction: a = frac * 2^exp, frac in [0.5,1)
    frac, exp = torch.frexp(a)  # exp is int-like tensor
    # Convert to IEEE-style normalized mantissa in [1,2): mant = 2*frac
    mant = frac * 2.0
    e = exp - 1  # now floor(log2(a))

    # Normal exponent field range for FP8 is limited; emulate a reasonable clamp
    # (This is approximate; NVIDIA uses specific FP8 variants.)
    e_clamped = e.clamp(-6, 8)  # keeps within range that covers up to ~448

    base = torch.ldexp(torch.ones_like(a), e_clamped)  # 2^e
    mant_norm = (a / base)  # ~[1,2)

    # Quantize mantissa to 3 bits: 1.xxx with 3 fractional bits
    mant_q = torch.round((mant_norm - 1.0) * (2**m_bits)) / (2**m_bits) + 1.0

    # Handle rounding overflow (e.g., mant_q becomes 2.0)
    carry = mant_q >= 2.0
    mant_q = torch.where(carry, mant_q * 0.5, mant_q)
    e2 = torch.where(carry, e_clamped + 1, e_clamped)

    # Reconstruct and clamp again
    out_nz = mant_q * torch.ldexp(torch.ones_like(a), e2)
    out_nz = out_nz.clamp_max(448.0)

    out = torch.where(nz, out_nz, out)
    return sign * out


# -----------------------------
# NVFP4-like quantization
# -----------------------------
@torch.no_grad()
def nvfp4(tensor: torch.Tensor, block_size: int = 16, return_meta: bool = False):
    """
    NVFP4-like quantization (emulated) with:
      Step 1: per-tensor FP32 scale (global_sf)
      Step 2: per-block FP8 E4M3 scale (s_fp8), block_size=16
      Step 3: per-element FP4 E2M1 values (q_fp4)

    Returns dequantized tensor (verification). Optionally returns (global_sf, s_fp8).
    """
    if tensor.numel() == 0:
        return (tensor, None) if return_meta else tensor

    orig_shape = tensor.shape
    x = tensor.reshape(-1).float()
    n = x.numel()

    # Pad to multiple of block_size (avoid torch.cat)
    r = n % block_size
    if r != 0:
        pad = block_size - r
        x_pad = torch.empty(n + pad, device=x.device, dtype=x.dtype)
        x_pad[:n].copy_(x)
        x_pad[n:].zero_()
        x = x_pad

    xb = x.view(-1, block_size)  # [num_blocks, block_size]

    # -------------------------
    # Step 1: Per-tensor scaling (FP32)
    # Choose global_sf so that block scales fit FP8 range (~448) with qTypeMax=6
    # TensorRT doc expresses scale_fp8 ~ cast_fp8( amax(block) / (6 * globalSf) ).  [oai_citation:5‡NVIDIA Docs](https://docs.nvidia.com/deeplearning/tensorrt/latest/inference-library/work-quantized-types.html?utm_source=chatgpt.com)
    # So set globalSf = max_block_amax / (6 * 448) to keep inside range.
    # -------------------------
    block_amax = xb.abs().amax(dim=1, keepdim=True)  # [B,1]
    max_block_amax = block_amax.max()
    denom = 6.0 * 448.0
    global_sf = torch.where(max_block_amax > 0, max_block_amax / denom, torch.tensor(1.0, device=x.device))
    global_sf = global_sf.clamp_min(1e-12)

    # -------------------------
    # Step 2: Per-block scaling (FP8 E4M3)
    # Unquantized block scale in "FP8 domain"
    # -------------------------
    s = block_amax / (6.0 * global_sf)  # [B,1], ideally <= 448
    s_fp8 = fp8_e4m3_quant_dequant(s)   # emulate storing scale in FP8 E4M3

    # Avoid divide-by-zero for all-zero blocks
    s_fp8 = s_fp8.clamp_min(1e-12)

    # -------------------------
    # Step 3: Quantize values to FP4 E2M1 under combined scale
    # q_fp4 = cast_fp4( x / (global_sf * s_fp8) )
    # dequant = global_sf * s_fp8 * deq_fp4(q_fp4)
    # -------------------------
    y = xb / (global_sf * s_fp8)
    yq = fp4_e2m1_quant_dequant(y)
    deq = yq * (global_sf * s_fp8)

    # Unpad + reshape
    deq = deq.view(-1)[:n].view(orig_shape).to(tensor.dtype)

    if return_meta:
        # Return dequant + (global_sf scalar, per-block fp8 scale shaped [num_blocks,1])
        return deq, {"global_sf": global_sf.to(tensor.dtype), "block_scales_fp8": s_fp8.to(tensor.dtype)}
    return deq

In [ ]:
x = torch.randn(128, 354)
x_nvfp4 = nvfp4(x, block_size=16)
x_quartz = quartz(x, quant_block_size=32, lzs_block_size=16, mode="lzs")

print("Error NVFP4: ", torch.norm(x - x_nvfp4, p='fro').item())
print("Error QUARTZ: ", torch.norm(x - x_quartz, p='fro').item())

In [ ]:
w.min(), w.max(), w.mean(), w.var()

In [ ]:
rs_w.min(), rs_w.max(), rs_w.mean(), rs_w.var()

In [ ]:
dset = "MJHQ"
t = 0
input_acts_org = act[dset]["org"][t]["input_acts"]
input_acts_rs = act[dset]["reverse"][t]["input_acts"]
output_acts_org = act[dset]["org"][t]["output_acts"]
output_acts_rs = act[dset]["reverse"][t]["output_acts"]

In [ ]:
input_acts_org.min(), input_acts_org.max(), input_acts_org.mean(), input_acts_org.var()

In [ ]:
output_acts_org.min(), output_acts_org.max(), output_acts_org.mean(), output_acts_org.var()

In [ ]:
input_acts_rs.min(), input_acts_rs.max(), input_acts_rs.mean(), input_acts_rs.var()

In [ ]:
output_acts_rs.min(), output_acts_rs.max(), output_acts_rs.mean(), output_acts_rs.var()

In [ ]:
smooth_scale.min()

### Tensor selection for RS vs non-RS comparison

We build `data` from the cached weights/activations and then select the input activations to
`transformer_blocks.11.attn.to_q` because that layer's output is exactly `XW` for the same weight
matrix `W`. This makes `||XW - Q(X)Q(W)||_F` a direct measure of quantization error.

We fix dataset (`MJHQ`), timestep, and batch index to avoid mixing distributions, and we use the
same evenly spaced token subset for both modes to isolate the effect of reverse smoothing. The
token subsample keeps the matmul tractable while still covering the full sequence.


In [ ]:
data = {
    "weights": {"org": w, "rs": rs_w},
    "activations": act,
}

dataset = "MJHQ"
timestep = 0
batch_index = 0
token_stride = 32  # 4096 / 32 = 128 tokens

x_org_full = data["activations"][dataset]["org"][timestep]["input_acts"][batch_index, 0]
x_rs_full = data["activations"][dataset]["reverse"][timestep]["input_acts"][batch_index, 0]

token_idx = torch.arange(0, x_org_full.shape[0], token_stride)
x_org = x_org_full[token_idx].float()
x_rs = x_rs_full[token_idx].float()
w_org = data["weights"]["org"].float()
w_rs = data["weights"]["rs"].float()

In [ ]:
@torch.no_grad()
def quartz_lastdim(
    tensor: torch.Tensor,
    quant_block_size: int = 32,
    lzs_block_size: int = 16,
    mode: str = "lzs",
) -> torch.Tensor:
    if tensor.numel() == 0:
        return tensor
    if quant_block_size % lzs_block_size != 0:
        raise ValueError(
            f"quant_block_size ({quant_block_size}) must be a multiple of lzs_block_size ({lzs_block_size})"
        )

    orig_shape = tensor.shape
    x = tensor.reshape(-1, orig_shape[-1])
    bsz, width = x.shape

    r = width % quant_block_size
    if r != 0:
        pad = quant_block_size - r
        x_pad = torch.empty(bsz, width + pad, device=x.device, dtype=x.dtype)
        x_pad[:, :width].copy_(x)
        x_pad[:, width:].zero_()
        x = x_pad
        width = width + pad

    x = x.view(bsz, -1, quant_block_size)
    x_fp = x.float()

    if mode == "mx":
        scales = x_fp.abs().amax(dim=2, keepdim=True) / 127.0
    elif mode == "lzs":
        scales = (x_fp.amax(dim=2, keepdim=True) - x_fp.amin(dim=2, keepdim=True)) / 64.0
    else:
        raise ValueError(f"Unknown mode {mode} for QUARTZ quantization")

    scales = scales.clamp_min_(1e-12)
    q = torch.round(x_fp / scales).clamp_(-128, 127)

    q = q.view(-1, lzs_block_size)
    q = lzs(q, bits=4)
    q = q.view(bsz, -1, quant_block_size)

    deq = (q * scales).to(tensor.dtype)
    if r != 0:
        deq = deq.view(bsz, -1)[:, : orig_shape[-1]]
    return deq.view(orig_shape)


@torch.no_grad()
def quartz_matmul_error(
    x: torch.Tensor,
    w: torch.Tensor,
    group_size: int,
    lzs_block_size=None,
    mode: str = "lzs",
    y_ref=None,
) -> float:
    if lzs_block_size is None:
        lzs_block_size = min(16, group_size)
    if group_size % lzs_block_size != 0:
        raise ValueError(
            f"group_size ({group_size}) must be a multiple of lzs_block_size ({lzs_block_size})"
        )

    if y_ref is None:
        y_ref = x @ w

    x_q = quartz_lastdim(x, group_size, lzs_block_size, mode=mode)
    w_q = quartz_lastdim(w, group_size, lzs_block_size, mode=mode)
    y_q = x_q @ w_q

    return torch.norm(y_ref - y_q, p="fro").item()


In [ ]:
group_sizes = [1, 4, 8, 16, 32, 64, 128]

y_org = x_org @ w_org
y_rs = x_rs @ w_rs

errors_org = []
errors_rs = []

for g in group_sizes:
    lzs_block = min(16, g)
    errors_org.append(quartz_matmul_error(x_org, w_org, g, lzs_block_size=lzs_block, y_ref=y_org))
    errors_rs.append(quartz_matmul_error(x_rs, w_rs, g, lzs_block_size=lzs_block, y_ref=y_rs))

errors_org, errors_rs


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

colors = {"org": "#0072B2", "rs": "#D55E00"}  # Okabe-Ito palette

fig, ax = plt.subplots(figsize=(7.2, 4.4), dpi=140)
ax.plot(group_sizes, errors_org, marker="o", lw=2.2, color=colors["org"], label="No RS")
ax.plot(group_sizes, errors_rs, marker="s", lw=2.2, ls="--", color=colors["rs"], label="RS")

ax.set_xscale("log", base=2)
ax.set_xticks(group_sizes)
ax.xaxis.set_major_formatter(mticker.ScalarFormatter())
ax.xaxis.set_minor_formatter(mticker.NullFormatter())

ax.set_xlabel("Group size")
ax.set_ylabel(r"$\|XW - Q(X)Q(W)\|_F$")
ax.set_title("QUARTZ quantization error vs group size")
ax.grid(True, axis="y", linestyle="--", alpha=0.45)
ax.legend(frameon=False)

plt.tight_layout()
plt.show()

### Smoothing scale and alpha grid search (nvfp4 extractions)

These helpers compute a per-input-channel smooth scale from input activations and weights
at transformer_blocks.11.attn.to_q, then run a simple alpha grid search using QUARTZ
quantization to pick the alpha that minimizes ||XW - Q(X)Q(W)||_F. Reverse smoothing
uses the inverse scaling (X * s, W / s) but shares the same scale formula.


In [ ]:
import torch

layer_name = "transformer_blocks.11.attn.to_q"

if "act_org" not in globals():
    act_org = torch.load(
        "final_result/nvfp4/extractions/activations_during_generation.pt",
        weights_only=False,
    )
if "w_org" not in globals():
    w_org = torch.load(
        "final_result/nvfp4/extractions/weights_before_smooth.pt",
        weights_only=False,
    )


In [ ]:
@torch.no_grad()
def extract_layer_inputs(
    act_cache,
    layer,
    dataset="MJHQ",
    timestep=0,
    max_samples=None,
):
    key = f"{layer}.input"
    raw = act_cache[key][-1]
    selected = []
    for i, tensor in enumerate(raw):
        dset = "MJHQ" if i < 64 else "sDCI"
        t = i % 4
        if dset == dataset and t == timestep:
            selected.append(tensor)
            if max_samples is not None and len(selected) >= max_samples:
                break
    if not selected:
        raise ValueError(f"No activations found for dataset={dataset}, timestep={timestep}")
    return torch.stack(selected, dim=0)


@torch.no_grad()
def flatten_acts(x: torch.Tensor) -> torch.Tensor:
    return x.reshape(-1, x.shape[-1])


@torch.no_grad()
def compute_span(x: torch.Tensor, mode: str = "abs_max") -> torch.Tensor:
    if mode == "abs_max":
        return x.abs().amax(dim=0)
    if mode == "rms":
        return x.pow(2).mean(dim=0).sqrt()
    raise ValueError(f"Unknown span mode: {mode}")


@torch.no_grad()
def smooth_scale_from_acts_weights(
    x: torch.Tensor,
    w: torch.Tensor,
    alpha: float,
    beta: float,
    span_mode: str = "abs_max",
    eps: float = 1e-12,
) -> torch.Tensor:
    x_span = compute_span(x, mode=span_mode)
    w_span = compute_span(w, mode=span_mode)
    scale = (x_span.clamp_min(eps).pow(alpha) / w_span.clamp_min(eps).pow(beta))
    return scale.nan_to_num(1.0, posinf=1.0, neginf=1.0)


@torch.no_grad()
def apply_smoothing(x: torch.Tensor, w: torch.Tensor, scale: torch.Tensor):
    scale_x = scale.view(1, -1)
    x_sm = x / scale_x
    w_sm = w * scale
    return x_sm, w_sm


@torch.no_grad()
def apply_reverse_smoothing(x: torch.Tensor, w: torch.Tensor, scale: torch.Tensor):
    scale_x = scale.view(1, -1)
    x_sm = x * scale_x
    w_sm = w / scale
    return x_sm, w_sm


@torch.no_grad()
def smooth_scale_for_layer(
    act_cache,
    weight_cache,
    layer,
    *,
    dataset="MJHQ",
    timestep=0,
    max_samples=None,
    token_stride=32,
    alpha=0.5,
    beta_mode="one_minus_alpha",
    span_mode="abs_max",
):
    inputs = extract_layer_inputs(
        act_cache, layer, dataset=dataset, timestep=timestep, max_samples=max_samples
    )
    inputs = inputs[:, 0]
    inputs = inputs[:, ::token_stride, :]
    x = flatten_acts(inputs).float()
    w = weight_cache[layer]["weight"].float()
    beta = 1.0 - alpha if beta_mode == "one_minus_alpha" else float(beta_mode)
    scale = smooth_scale_from_acts_weights(x, w, alpha=alpha, beta=beta, span_mode=span_mode)
    return scale, x, w


In [ ]:
@torch.no_grad()
def grid_search_alpha(
    x: torch.Tensor,
    w: torch.Tensor,
    *,
    mode: str = "smooth",
    span_mode: str = "abs_max",
    num_grids: int = 20,
    alpha_candidates=None,
    beta_mode="one_minus_alpha",
    quant_block_size: int = 32,
    lzs_block_size: int = 16,
):
    if alpha_candidates is None:
        if mode == "reverse":
            alpha_candidates = [i / num_grids for i in range(-num_grids, num_grids + 1) if i != 0]
        else:
            alpha_candidates = [i / num_grids for i in range(1, num_grids + 1)]

    y_ref = x @ w.t()
    results = []
    for alpha in alpha_candidates:
        beta = 1.0 - alpha if beta_mode == "one_minus_alpha" else float(beta_mode)
        scale = smooth_scale_from_acts_weights(x, w, alpha=alpha, beta=beta, span_mode=span_mode)
        if mode == "reverse":
            x_sm, w_sm = apply_reverse_smoothing(x, w, scale)
        elif mode == "smooth":
            x_sm, w_sm = apply_smoothing(x, w, scale)
        else:
            raise ValueError(f"Unknown mode: {mode}")
        x_q = quartz_lastdim(x_sm, quant_block_size, lzs_block_size, mode="lzs")
        w_q = quartz_lastdim(w_sm, quant_block_size, lzs_block_size, mode="lzs")
        err = torch.norm(y_ref - (x_q @ w_q.t()), p="fro").item()
        results.append((alpha, err))
    best = min(results, key=lambda it: it[1])
    return results, best


In [ ]:
scale, x_flat, w_mat = smooth_scale_for_layer(
    act_org,
    w_org,
    layer_name,
    dataset="MJHQ",
    timestep=0,
    max_samples=8,
    token_stride=32,
    alpha=0.5,
    beta_mode="one_minus_alpha",
    span_mode="abs_max",
)

x_sm, w_sm = apply_smoothing(x_flat, w_mat, scale)
x_rev, w_rev = apply_reverse_smoothing(x_flat, w_mat, scale)

scale.min().item(), scale.max().item()


In [ ]:
smooth_results, smooth_best = grid_search_alpha(
    x_flat,
    w_mat,
    mode="smooth",
    span_mode="abs_max",
    num_grids=20,
    beta_mode="one_minus_alpha",
    quant_block_size=32,
    lzs_block_size=16,
)

reverse_results, reverse_best = grid_search_alpha(
    x_flat,
    w_mat,
    mode="reverse",
    span_mode="abs_max",
    num_grids=20,
    beta_mode="one_minus_alpha",
    quant_block_size=32,
    lzs_block_size=16,
)

smooth_best, reverse_best


In [ ]:
import torch
from diffusers import Transformer2DModel, PixArtSigmaPipeline

# Set fixed seed for reproducibility
torch.manual_seed(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weight_dtype = torch.float16

pipe = PixArtSigmaPipeline.from_pretrained(
    "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS", 
    torch_dtype=weight_dtype,
    use_safetensors=True,
)
pipe = pipe.to(device)

In [ ]:
from deepcompressor.app.diffusion.investigate import ActivationModifier, ActivationExtractor

# Define skip patterns based on the YAML config
skip_patterns = [
    "embed",              # embedding layers
    "proj_in",            # transformer_proj_in
    "proj_out",           # transformer_proj_out
    "adaln_single",       # transformer_norm / transformer_add_norm
    "caption_projection", # embed
    "time_embed",         # resblock_time_proj
]

def get_allowed_layers(model, skip_patterns: list[str]) -> set[str]:
    """Get all Linear layer names except those matching skip patterns."""
    allowed = set()
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            # Check if this layer should be skipped
            should_skip = any(pattern in name for pattern in skip_patterns)
            if not should_skip:
                allowed.add(name)
    return allowed

# Get the transformer model from the pipeline
transformer = pipe.transformer

# Get allowed layers (all Linear layers except skipped ones)
allowed_layers = get_allowed_layers(transformer, skip_patterns)
allowed_layers = [name for name in allowed_layers if 'transformer_blocks.0' in name]

print(f"Total allowed layers: {len(allowed_layers)}")
print("Allowed layers:", sorted(allowed_layers)[:10], "...")  # Print first 10

In [ ]:
# Create and register the modifier
torch.manual_seed(12345)
prompt = "ultra realistic, summer, jungle, uranian blue, ice blue, light green, melon, salmon pink, flowers, leaves, white background "
# image = pipe(prompt).images[0]
# image.save(f"./figs/tau_removed/fp.png")
extractor = ActivationExtractor(
    transformer,
    allowed_layers=allowed_layers,
)
extractor.register_hooks()

# Run inference with modified activations
image = pipe(prompt).images[0]
extractor.save_activations(suffix="pixart")
extractor.remove_hooks()

In [ ]:
extractor.save_activations()

In [ ]:
# Create and register the modifier
torch.manual_seed(12345)
prompt = "ultra realistic, summer, jungle, uranian blue, ice blue, light green, melon, salmon pink, flowers, leaves, white background "
# image = pipe(prompt).images[0]
# image.save(f"./figs/tau_removed/fp.png")
for ratio in [0.25]:
    torch.manual_seed(12345)
    modifier = ActivationModifier(
        transformer,
        allowed_layers=allowed_layers,
        zero_small_activations=True,
        small_threshold_ratio=ratio,
        mode="oneminustau",
    )
    modifier.register_hooks()

    # Run inference with modified activations
    image = pipe(prompt).images[0]
    image.save(f"./figs/tau_removed/outlier_{1-ratio}.png")

    # Clean up hooks after inference
    modifier.remove_hooks()

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

base_path = "/home/work/workspace/dhkim2810/qtbench/examples/diffusion/baselines/torch.bfloat16/flux.1-schnell/fmeuler4-g0/samples"
target_path = "/home/work/workspace/dhkim2810/qtbench/examples/diffusion/runs/diffusion/flux.1/flux.1-schnell/w.4-x.4-y.16/w.sint4-x.sint4.u-y.bf16/w.v32.bf16-x.v32.bf16-y.tnsr.bf16/w.static/shift-skip.x.[[w]+tan+tn].w.[e+rs+rtp+s+tpi+tpo]-qdiff.128-t4.g0-s5000/no_inliner_1/samples"

dset = "MJHQ" # or "MJHQ"
if dset == "MJHQ":
    base_path += "/MJHQ/MJHQ-5000"
    target_path += "/MJHQ/MJHQ-16"
else:
    base_path += "/DCI/sDCI-5000"
    target_path += "/DCI/sDCI-16"

base_imgs = os.listdir(base_path)
filtered_imgs = os.listdir(target_path)

# Find common images in both directories
common_imgs = set(base_imgs).intersection(set(filtered_imgs))
print(f"Found {len(common_imgs)} common images.")

In [ ]:
img_idx = 0
tau = 2
target_path = target_path.replace("no_inliner_1", f"no_inliner_{tau}")
# Show images side by side
for img_name in sorted(common_imgs):
    base_img = Image.open(os.path.join(base_path, img_name))
    target_img = Image.open(os.path.join(target_path, img_name))

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(base_img)
    axes[0].set_title("Base Image")
    axes[0].axis("off")

    axes[1].imshow(target_img)
    axes[1].set_title("Target Image")
    axes[1].axis("off")

    plt.suptitle(f"Image: {img_name}")
    plt.show()

    img_idx += 1
    if img_idx >= 5:  # Display only first 5 images for brevity
        break

# Quantization Type Case Study

1. SINT4
2. UINT4
3. CINT4

In [ ]:
import torch
from typing import Tuple

def qint8(x: torch.Tensor, s: torch.Tensor | None, z: torch.Tensor | None, qtype: str = "sym") -> Tuple[torch.Tensor, torch.Tensor]:
    if qtype == "sym":
        s = x.abs().max() / 127.0
        z = torch.zeros_like(s)
        qmin, qmax = -128, 127
    elif qtype == "asym":
        s = (x.max() - x.min()) / 255.0
        z = torch.round(-x.min() / s)
        qmin, qmax = 0, 255
    elif qtype == "custom":
        s = (x.max() - x.min()) / 128.0
        z = torch.zeros_like(s)
        qmin, qmax = -128, 127

    x_scaled = x / s
    x_rounded = torch.round(x_scaled) + z
    x_clamped = x_rounded.clamp(qmin, qmax)
    x_dequant = (x_clamped - z) * s

    return x_dequant, x_rounded

q_err = lambda x, x_dq : torch.norm(x - x_dq, p='fro').item()

In [ ]:
x = torch.randn(128)
s = None
z = None

target_var = 0.1
target_mean = 2.1

x = x * (target_var / x.var()).sqrt()
x = x - x.mean() + target_mean

In [ ]:
print(f"X mean/var before quant: {x.mean().item():.6f} / {x.var().item():.6f}")

In [ ]:
x_dq_sym, x_int_sym = qint8(x, s, z, qtype="sym")
q_err(x, x_dq_sym)

In [ ]:
x_dq_asym, x_int_asym = qint8(x, s, z, qtype="asym")
q_err(x, x_dq_asym)

In [ ]:
x_dq_c, x_int_c = qint8(x, s, z, qtype="custom")
q_err(x, x_dq_c)

In [ ]:
x_int_c

In [ ]:
import matplotlib.pyplot as plt

x_density, bins_density = torch.histogram(x, bins=256, range=(-4, 4), density=True)
x_dq_sym_density, _ = torch.histogram(x_dq_sym, bins=256, range=(-4, 4), density=True)
x_dq_asym_density, _ = torch.histogram(x_dq_asym, bins=256, range=(-4, 4), density=True)
x_dq_c_density, _ = torch.histogram(x_dq_c, bins=256, range=(-4, 4), density=True)
plt.figure(figsize=(10, 6))
plt.bar(bins_density[:-1].numpy(), x_density.numpy(), width=(bins_density[1]-bins_density[0]).item(), alpha=0.5, label='Original', color='blue')
plt.bar(bins_density[:-1].numpy(), x_dq_sym_density.numpy(), width=(bins_density[1]-bins_density[0]).item(), alpha=0.5, label='Quantized Sym', color='orange')
plt.bar(bins_density[:-1].numpy(), x_dq_asym_density.numpy(), width=(bins_density[1]-bins_density[0]).item(), alpha=0.5, label='Quantized Asym', color='green')
# plt.bar(bins_density[:-1].numpy(), x_dq_c_density.numpy(), width=(bins_density[1]-bins_density[0]).item(), alpha=0.5, label='Quantized Custom', color='red')
plt.legend()
plt.title('Histogram of Original and Quantized Values')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import numpy as np

x = np.random.randint(0, 32, size=(32,), dtype=np.uint8)
x_reduce = np.bitwise_or.reduce(x)
flag = max(np.floor(np.log2(x_reduce)) - 3, np.uint8(0)).astype(np.uint8)
print(x_reduce, flag)
x_compressed = x >> flag
x, x_compressed

In [ ]:
np.uint8(0)

dict_keys(['transformer_blocks.0.attn1.to_q', 'transformer_blocks.0.attn1.to_k', 'transformer_blocks.0.attn1.to_v', 'transformer_blocks.0.attn1.to_out.0', 'transformer_blocks.0.attn2.to_q', 'transformer_blocks.0.attn2.to_k', 'transformer_blocks.0.attn2.to_v', 'transformer_blocks.0.attn2.to_out.0', 'transformer_blocks.0.ff.net.0.proj', 'transformer_blocks.0.ff.net.2'])

In [12]:
xt[0].shape

torch.Size([2, 4096, 1152])

In [13]:
xt[0].mean(), xt[0].var(), xt[0].min(), xt[0].max()

(tensor(0.0085, dtype=torch.float16),
 tensor(0.0368, dtype=torch.float16),
 tensor(-3.1660, dtype=torch.float16),
 tensor(2.7305, dtype=torch.float16))

36.0